# FMDataAPI Sample Results

by Masayuki Nii (nii@msyk.net)

FMDataAPI is a class developed by PHP to access FileMaker database with FileMaker Data API.

The repository is https://github.com/msyk/FMDataAPI.

API Document is http://inter-mediator.info/FMDataAPI/namespaces/INTERMediator.FileMakerServer.RESTAPI.html.

The identifier of Composer is "inter-mediator/fmdataapi".

Ths notebook aims to show the results of FMDataAPI in short hand. You don't have to prepare PHP even FileMaker Server because you can see all results with codes in this notebook.

## Preparing to Use FMDataAPI in PHP way

First of all, the FMDataAPI.php file has to be included. All classes are defined in it. Of course, you can specify partial or full path, or composer based class path resolving.

In [1]:
include_once "FMDataAPI.php";

1

For your convenience, the main class name FMDataAPI is defined at the current namespace.

In [2]:
use INTERMediator\FileMakerServer\RESTAPI\FMDataAPI as FMDataAPI;

## Getting Access to FileMaker DB
Instanticate the class FMDataAPI with database name, user name, password and host.Although the port number and protocol can be set in parameters of constractor, these parameters can be omitted with default values.

In [3]:
$fmdb = new FMDataAPI("TestDB", "web", "password", "localserver");

INTERMediator\FileMakerServer\RESTAPI\FMDataAPI {#200}

### Very Simple Query, Getting All Records
The FMDataAPI has the property as the same name of layout. This sample database has the 'person_layout' layout, so '$fmdb->person_layout' refers FMLayout object fo the proxy of the layout. FMLayout class has the 'query' method and returns FileMakerRelation class's object. The condition spefied in parameter is same as FileMaker's Find Record API. This example means querying all record from the "person_layout" layout.

In [4]:
$result = $fmdb->person_layout->query();

INTERMediator\FileMakerServer\RESTAPI\Supporting\FileMakerRelation {#192}

The 'httpStatus()' method returns the HTTP status code in the latest response.

In [5]:
echo "HTTP Status: {$fmdb->httpStatus()}";

HTTP Status: 200


The following two methods return the error code and message of the latest API call which is submitted in query() method. You can check API calling succeed or fail if error code is or isn't 0 every after API calling methods.

In [6]:
echo "Error Code: {$fmdb->errorCode()}";
echo "Error Message: {$fmdb->errorMessage()}";

Error Code: 0


Error Message: 


### Accessing Queried Data
The FileMakerRelation class implements the Iterator interface and it can repeat with 'foreach.' The \$record also refers a FileMakerRelation object but it is for single record. This layout has fields as like 'id', 'name', 'mail' and so on, and the field name can be handle as a property name of the the record referring with \$record. The 'person' table in database has 3 records and you can see 3 outs below. The recordId or redId is required to update the record, and it can get by the getRecordId method.

In [7]:
if (!is_null($result)) {
    foreach ($result as $record) {
       echo "[id]{$record->id}, [name]{$record->name}, [mail]{$record->mail}";
       echo "__[recordId = {$record->getRecordId()}]";
    }
}

[id]1, [name]Masayuki Nii, [mail]Dog


__[recordId = 1]


[id]2, [name]Someone, [mail]msyk@msyk.net


__[recordId = 2]


[id]3, [name]Anyone, [mail]Dog


__[recordId = 3]


### Accessing to Portal Data
A portal name property returns records of portal as FileMakerRelation object. You can repeat with foreach for the portal records.

Technically portal field has to be refered as "contact_to::id" but it can be an indentifier in PHP. In this case you can call field method as like 'field("summary", "contact_to").' If the field belongs to the table occurrence for the portal, you can refer the field as like '$item->id.' If the field belongs to another table occurrence, you have to call the 'field()' method.

If the object name of the portal is blank, it can be referred as the table occurrence name. If the object name is specified, you have to access with the object name and it means you have to call 'field()' method to get the value.

In [8]:
if (!is_null($result)) {
    foreach ($result as $record) {
       echo "id: {$record->id}, name: {$record->name}, mail: {$record->mail}";
       $contacts = $record->Contact;
       foreach ($contacts as $item) {
           $id = $item->field("id", "contact_to");
           $summary = $item->field("summary", "contact_to");
           echo "__[PORTAL(contact_to)][id]{$id}, [summary]{$summary}";
       }
    }
}

id: 1, name: Masayuki Nii, mail: Dog


__[PORTAL(contact_to)][id]1, [summary]Telephone


__[PORTAL(contact_to)][id]2, [summary]Meetings


__[PORTAL(contact_to)][id]3, [summary]Mail


id: 2, name: Someone, mail: msyk@msyk.net


__[PORTAL(contact_to)][id]4, [summary]Calling


__[PORTAL(contact_to)][id]5, [summary]Telephone


id: 3, name: Anyone, mail: Dog


__[PORTAL(contact_to)][id]6, [summary]Meeting


__[PORTAL(contact_to)][id]7, [summary]Mail etcsss


The FileMakerRelation object from 'query()' method can be accessed as like the 'cursor' style repeating.
The 'count()' method returns the number of records in response. The variable $result referes current record and you can get the field value with the propaty having the same field name.
The portal can be done with same way. The 'next()' method steps forward the pointer of current record.
Before examining the cursor looping, the pointer has to move to the first record with the rewind() method.

In [9]:
$result->rewind();
for ($i = 0; $i < $result->count(); $i++) {
    echo "[id]{$result->id}, [name]{$result->name}, [mail]{$result->mail}";
    $contacts = $result->Contact;
    $contacts->rewind();
    for ($j = 0; $j < $contacts->count(); $j++) {
        $id = $contacts->field("id", "contact_to");
        $summary = $contacts->field("summary", "contact_to");
        echo "__[PORTAL(contact_to)][id]{$id}, [summary]{$summary}";
        $contacts->next();
    }
    $histories = $result->History;
    $histories->rewind();
    for ($j = 0; $j < $histories->count(); $j++) {
        $std = $histories->field("startdate", "history_to");
        $endd = $histories->field("enddate", "history_to");
        echo "__[PORTAL(history_to)][startdate]{$std}, [enddate]{$endd}";
        $histories->next();
    }
    $result->next();
}

[id]1, [name]Masayuki Nii, [mail]Dog


__[PORTAL(contact_to)][id]1, [summary]Telephone


__[PORTAL(contact_to)][id]2, [summary]Meetings


__[PORTAL(contact_to)][id]3, [summary]Mail


__[PORTAL(history_to)][startdate]04/01/2001, [enddate]03/31/2003


...

__[PORTAL(contact_to)][id]4, [summary]Calling


__[PORTAL(contact_to)][id]5, [summary]Telephone


[id]3, [name]Anyone, [mail]Dog


__[PORTAL(contact_to)][id]6, [summary]Meeting


__[PORTAL(contact_to)][id]7, [summary]Mail etcsss


### Sort Fields
The second parameter of query method specifies fields for sorting with array of array data. The innner array has one or two elements with the field name and the dicrection as below. This menas ordering by the id field decendly.

In [10]:
$result = $fmdb->person_layout->query(null, [["id", "descend"]]);
if (!is_null($result)) {
    foreach ($result as $record) {
       echo "[id]{$record->id}, [name]{$record->name}, [mail] {$record->mail}";
    }
}

[id]3, [name]Anyone, [mail] Dog


[id]2, [name]Someone, [mail] msyk@msyk.net


[id]1, [name]Masayuki Nii, [mail] Dog


### Start Record and Limit Records
The third and fourth parameter of the query method specify the start record number and the limit number of record. The following query means query 5 records from 20th record with ordered by the f3 fields.

In [11]:
$result = $fmdb->postalcode->query(null, [["f3"]], 20, 5);
if (!is_null($result)) {
    foreach ($result as $record) {
       echo "[postal code]{$record->f3}, [place]{$record->f7}{$record->f8}{$record->f9}";
    }
}

[postal code]1000400, [place]東京都新島村以下に掲載がない場合


[postal code]1000401, [place]東京都新島村若郷


[postal code]1000402, [place]東京都新島村本村


[postal code]1000511, [place]東京都新島村式根島


[postal code]1000601, [place]東京都神津島村神津島村一円


### Query with Condtion
The 'query()' method can have several parameters. The first parameter specifies condtions with array of array. Simply key is a field name and value is the value. The following condition means just query the id field is "1".

In [12]:
$result = $fmdb->person_layout->query([["id" => "1"]]);
if (!is_null($result)) {
    foreach ($result as $record) {
       echo "[id]{$record->id}, [name]{$record->name}, [mail] {$record->mail}";
    }
}

[id]1, [name]Masayuki Nii, [mail] Dog


Next query means character 中 contains in the f9 field.

In [13]:
$result = $fmdb->postalcode->query([["f9" => "中"]], null, 1, 3);
if (!is_null($result)) {
    foreach ($result as $record) {
       echo "[postal code]{$record->f3}, [place]{$record->f7}{$record->f8}{$record->f9}";
    }
}

[postal code]1030008, [place]東京都中央区日本橋中洲


[postal code]1610035, [place]東京都新宿区中井


[postal code]1610032, [place]東京都新宿区中落合


Next query means the f8 field is just 中央区 AND character 中 contains in the f9 field.

In [14]:
$result = $fmdb->postalcode->query([["f8" => "=中央区", "f9" => "中"]], null, 1, 3);
if (!is_null($result)) {
    foreach ($result as $record) {
       echo "[postal code]{$record->f3}, [place]{$record->f7}{$record->f8}{$record->f9}";
    }
}

[postal code]1030008, [place]東京都中央区日本橋中洲


Next query means the character 中 contains in the f8 OR f9 field.

In [15]:
$result = $fmdb->postalcode->query([["f8" => "中"], ["f9" => "中"]], null, 1, 3);
if (!is_null($result)) {
    foreach ($result as $record) {
       echo "[postal code]{$record->f3}, [place]{$record->f7}{$record->f8}{$record->f9}";
    }
}

[postal code]1030000, [place]東京都中央区以下に掲載がない場合


[postal code]1040044, [place]東京都中央区明石町


[postal code]1040042, [place]東京都中央区入船


Next query means the f3 field is within the range from 170000 to 171000.

In [16]:
$result = $fmdb->postalcode->query([["f3" => "1700000...1710000"]], null, 1, 3);
if (!is_null($result)) {
    foreach ($result as $record) {
       echo "[postal code]{$record->f3}, [place]{$record->f7}{$record->f8}{$record->f9}";
    }
}

[postal code]1700000, [place]東京都豊島区以下に掲載がない場合


[postal code]1700014, [place]東京都豊島区池袋（１丁目）


[postal code]1700011, [place]東京都豊島区池袋本町


### Restricting to Specified Portals
The portal specification (the 5th parameter of query or the second parameter of getRecord) has to be an array with the object name of the portal not the table occurrence name. The person_layout has two portals named "Contact" and "History", but if the 5th parameter is specified, portal data in the result is just in the parameter.

In [17]:
$result = $fmdb->person_layout->query([["id" => "1"]], null, 1, -1, ["History"]);
if (!is_null($result)) {
    foreach ($result as $record) {
       echo "[id]{$record->id}, [name]{$record->name}, [mail]{$record->mail}";
       $histories = $record->History;
       foreach ($histories as $item) {
           $startdate = $item->field("startdate", "history_to");
           $enddate = $item->field("enddate", "history_to");
           echo "__[PORTAL(history_to)][startdate]{$startdate}, [enddate]{$enddate}";
       }
    }
}

[id]1, [name]Masayuki Nii, [mail]Dog


__[PORTAL(history_to)][startdate]04/01/2001, [enddate]03/31/2003


__[PORTAL(history_to)][startdate]04/01/2003, [enddate]03/31/2007


## Writing Operations
### Create Record
The 'create()' method creates a record with values in parameter.
The associated array of the parameter has to be a series of field name key and its value.

In [18]:
$recId = $fmdb->postalcode->create(["f3" => "field 3 data", "f7" => "field 7 data"]);

"4123"

The 'getRecord()' method query the record with the recordId of the parameter.
It returns the FileMakerRelation object and you can handle it with the return value from 'query()' method.

In [19]:
$result = $fmdb->postalcode->getRecord($recId);
if (!is_null($result)) {
    foreach ($result as $record) {
        echo "[f3]{$record->f3}, [f7]{$record->f7}, [f8]{$record->f8}";
    }
}

[f3]field 3 data, [f7]field 7 data, [f8]


### Update Record
The 'update()' method modifies fields in a record. You have to set parameters as the recordId of target record and associated array to specify the modified data.

In [20]:
$fmdb->postalcode->update($recId, ["f3" => "field 3 modifed", "f8" => "field 8 update"]);
$result = $fmdb->postalcode->getRecord($recId);
if (!is_null($result)) {
    foreach ($result as $record) {
       echo "[f3]{$record->f3}, [f7]{$record->f7}, [f8]{$record->f8}";
    }
}

[f3]field 3 modifed, [f7]field 7 data, [f8]field 8 update


## Delete Record
The 'delete()' method deletes the record specified by the parameter.

In [21]:
$fmdb->postalcode->delete($recId);

null

## Call Script
Some methods ex. query can execute a script with 6th paramter.

This example execute the "TextScript" script with the parameter "ok". The script finishes no error and returns value and you can detect it from the getScriptResult method.

In [22]:
$scriptSpec = ["script" => "TestScript", "script.param" => "ok"];
$result = $fmdb->person_layout->query(null, null, -1, 1, null, $scriptSpec);
if (!is_null($result)) {
    echo "Script Error: {$fmdb->person_layout->getScriptError()}";
    echo "Script Result: {$fmdb->person_layout->getScriptResult()}";
}

Script Error: 0


Script Result: It's over.


The "script" key's script executes just after querying. Otherwise the "script.prerequest" key's one does before querying. Errors and/or Result can be detect with methods described below.

In [23]:
$scriptSpec = ["script.prerequest" => "TestScript", "script.prerequest.param" => "ok"];
$result = $fmdb->person_layout->query(null, null, -1, 1, null, $scriptSpec);
if (!is_null($result)) {
    echo "Script Error: {$fmdb->person_layout->getScriptErrorPrerequest()}";
    echo "Script Result: {$fmdb->person_layout->getScriptResultPrerequest()}";

}

Script Error: 0


Script Result: It's over.


If any errors happens in the script, the error is not 0 as shown below.

In [24]:
$scriptSpec = ["script" => "TestScript", "script.param" => "not"];
$result = $fmdb->person_layout->query(null, null, -1, 1, null, $scriptSpec);
if (!is_null($result)) {
    echo "Script Error: {$fmdb->person_layout->getScriptError()}";
    echo "Script Result: {$fmdb->person_layout->getScriptResult()}";
}

Script Error: 102


Script Result: It's error.


If you don't specify any script information, error and result are both "blank" data.

In [25]:
$result = $fmdb->person_layout->query(null, null, -1, 1);
if (!is_null($result)) {
    echo "Script Error: {$fmdb->person_layout->getScriptError()}";
    echo "Script Result: {$fmdb->person_layout->getScriptResult()}";
}

Script Error: 


Script Result: 


## File Uploading
A new record is created in "testtable" table on the first statement of below.
Then the "testtable" table has a container filed "vc1". One image file is going to be uploaded to it.
The file path, record id and field name are required.

In [26]:
$recId = $fmdb->testtable->create();
$fmdb->testtable->uploadFile("samples/cat.jpg", $recId, "vc1");

null

What kind of data does the container field which inserted an image return? For example, the returned value is going to show as the value of the vc1 field as below. It'a kind of url, and it can get the content of the container field, and it means you can download with the getContainerData method. 

In [27]:
$result = $fmdb->testtable->getRecord($recId);
if(!is_null($result)) {
    foreach ($result as $record) {
        echo "vc1: {$record->vc1}";
        echo "<img src='data:image/jpeg;base64," . $record->getContainerData('vc1') . "'>";
    }
}

vc1: https://localhost/Streaming_SSL/MainDB/F18E548D6339DB444ED92BF13DE220A5F773E7E68607E29E388FBD6ECE1B5AEF.jpg?RCType=EmbeddedRCFileProcessor


Exception with message 'Error in creating cookie file. Failed to connect to localhost port 443: Connection refused'

## Batch Operations
If you call the 'startCommunication()' method, you can describe a series of database operation
calls. This means the authentication is going to be done at the 'startCommunication()' method, and the token is going to be shared with following statements. The 'endCommunication()' calls logout REST API call and invalidate the shared token.

In [28]:
$recIds = [];
$fmdb->postalcode->startCommunication();
$recIds[] = $fmdb->postalcode->create(["f3" => "field 3 data 1", "f7" => "field 7 data"]);
$recIds[] = $fmdb->postalcode->create(["f3" => "field 3 data 2", "f7" => "field 7 data"]);
$recIds[] = $fmdb->postalcode->create(["f3" => "field 3 data 3", "f7" => "field 7 data"]);
$recIds[] = $fmdb->postalcode->create(["f3" => "field 3 data 4", "f7" => "field 7 data"]);
$recIds[] = $fmdb->postalcode->create(["f3" => "field 3 data 5", "f7" => "field 7 data"]);
$recIds[] = $fmdb->postalcode->create(["f3" => "field 3 data 6", "f7" => "field 7 data"]);
$recIds[] = $fmdb->postalcode->create(["f3" => "field 3 data 7", "f7" => "field 7 data"]);
$fmdb->postalcode->endCommunication();
echo var_export($recIds, true);

array (
  0 => '4124',
  1 => '4125',
  2 => '4126',
  3 => '4127',
  4 => '4128',
  5 => '4129',
  6 => '4130',
)


## Parameter for Operations

You can turn off to throw an exception in case of error. You have to handle errors with checking result error.

In [29]:
$fmdb->setThrowException(false);

null

If you set to throwing exceptions, you can describe the try-catch statement in your code for error handling.

In [30]:
$fmdb->setThrowException(true);
$result = $fmdb->testtable->getRecord($recId);
try {
    foreach ($result as $record) {
        echo $record->not_exist_field;
    }
} catch(Exception $ex) {
    echo "[Exception]{$ex->getMessage()}";
}

[Exception]Field not_exist_field doesn't exist.


If you call with true, the debug mode is activated. Debug mode echos the contents of communication such as request and response.

In [31]:
$fmdb->setDebug(false);

null

If you call with true, the certificate from the server is going to verify. In case of self-signed one (usually default situation), you don't have to call this method.

In [32]:
//$fmdb->setCertValidating(true);